In [8]:
# Compute the Macaulay matrix in degree d of a system f = [f1,...,fs]
def mac_matrix(f,d,ring):
    mons0 = monomials(list(ring.gens()), [d+1 for i in range(len(ring.gens()))])
    mons = []
    for mon in mons0:
        if mon.degree() <= d:
            mons.append(mon)
    mons.sort()
    mons.reverse() # for reasons I don't understand, the .sort() method respects the chosen order but reverses it
    print(mons)
    col_labs = []
    for deg in range(d + 1):
        segment = []
        for poly in f:
            for mon in mons:
                label = ring(mon)*ring(poly)
                if label.degree() == deg:
                    segment.append(label)
        col_labs += segment
    return matrix([[label.monomial_coefficient(mon) for mon in mons] for label in col_labs])


# Takes an RREF Macaulay matrix as input and returns a list of the corresponding polynomials
def recover_polys(M, d, ring):
    mons0 = monomials(list(ring.gens()), [d+1 for i in range(len(ring.gens()))])
    mons = []
    for mon in mons0:
        if mon.degree() <= d:
            mons.append(mon)
    mons.sort()
    mons.reverse()
    B = []
    for row in M.rows():
        g = 0
        for (coeff,mon) in zip(row,mons):
            g += coeff*mon
        if g != 0:
            B.append(g)
    if B == []:
        B = [ring(0)]
    return Sequence(B) # Sage doesn't know how to work with the output list unless it is a "Sequence" of polynomials


# Now instead of using mac_matrix followed by recover_polys, we ask for the new polynomials directly:
def mac_basis(f,d,ring):
    mons0 = monomials(list(ring.gens()), [d+1 for i in range(len(ring.gens()))])
    mons = []
    for mon in mons0:
        if mon.degree() <= d:
            mons.append(mon)
    mons.sort()
    mons.reverse() # for reasons I don't understand, the .sort() method respects the chosen order but reverses it
    col_labs = []
    for deg in range(d + 1):
        segment = []
        for poly in f:
            for mon in mons:
                label = ring(mon)*ring(poly)
                if label.degree() == deg:
                    segment.append(label)
        col_labs += segment
    M = matrix([[label.monomial_coefficient(mon) for mon in mons] for label in col_labs]).rref()
    B = []
    for row in M.rows():
        g = 0
        for (coeff,mon) in zip(row,mons):
            g += coeff*mon
        if g != 0:
            B.append(g)
    if B == []:
        B = [ring(0)]
    return Sequence(B)


# Now define a function that calculates a Gröbner basis and the solving degree (per Caminata and Gorla, 2021)
def mac_grob(f,ring):
    test_deg = 0
    while True:
        B = mac_basis(f, test_deg, ring)
        print(test_deg, B, B.is_groebner(), ideal(B) == ideal(f))
        if B.is_groebner() and ideal(B) == ideal(f):
            return (test_deg,B)
        test_deg += 1
     
# Modified version
def sdeg(f, ring):
    d = 0
    while True:
        d_level_base = []
        mons = get_mons(d, ring)
        M = mac_matrix(f, d, ring)
        done = False
        while not done:
            rowsp = M.row_space()
            to_be_added = []
            for row in M.rref().rows():
                g = 0
                for (coeff, mon) in zip(row, mons):
                    g += coeff * mon
                max_test_deg = d - g.degree()
                if (g != 0) and (max_test_deg > 0):
                    for mon in get_mons(max_test_deg, ring):
                        test_p = mon*g
                        test_vec = vector([test_p.monomial_coefficient(moon) for moon in mons])
                        if test_vec not in rowsp:
                            to_be_added.append(test_vec)
            if to_be_added == []:
                done = True
                for row in M.rref().rows():
                    g = 0
                    for (coeff, mon) in zip(row, mons):
                        g += coeff * mon
                    if g != 0:
                        d_level_base.append(g)
                if d_level_base == []:
                    d_level_base = [ring(0)]
                check_1 = Sequence(d_level_base).is_groebner()
                check_2 = (ideal(d_level_base) == ideal(f))
                print(d, Sequence(d_level_base), check_1, check_2)
                if check_1 and check_2:
                    return (d, Sequence(d_level_base))
                d += 1
            else:
                new_rows = M.rows() + to_be_added
                M = Matrix(new_rows)

def test_rowsp(f, f_list, d, ring):
    mons = get_mons(d, ring)
    M = mac_matrix(f_list, d, ring)
    done = False
    while not done:
        rowsp = M.row_space()
        to_be_added = []
        for row in M.rref().rows():
            g = 0
            for (coeff, mon) in zip(row, mons):
                g += coeff * mon
            max_test_deg = d - g.degree()
            if (g != 0) and (max_test_deg > 0):
                for mon in get_mons(max_test_deg, ring):
                    test_p = mon*g
                    test_vec = vector([test_p.monomial_coefficient(moon) for moon in mons])
                    if test_vec not in rowsp:
                        to_be_added.append(test_vec)
        if to_be_added == []:
            done = True
        else:
            new_rows = M.rows() + to_be_added
            M = Matrix(new_rows)
    f_vec = vector([ring(f).monomial_coefficient(mon) for mon in mons])
    return f_vec in M.row_space()

def get_mons(d, ring, leq = True):
    if leq:
        mons = []
        for i in range(d+1):
            mons += [ring({tuple(a):1}) for a in WeightedIntegerVectors(i,[1 for gen in ring.gens()])]
    else:
        mons = [ring({tuple(a):1}) for a in WeightedIntegerVectors(d,[1 for gen in ring.gens()])]
    mons.sort()
    mons.reverse()
    return mons

def deg_fall(f, f_list, ring):
    test_deg = f.degree()
    while True:
        print(test_deg, time.strftime("%H:%M:%S", time.localtime()))
        if test_rowsp(f, f_list, test_deg, ring):
            return test_deg
        test_deg += 1
        
def lfd(f_list, ring):
    fall_degrees = []
    I = ideal(f_list)
    B = I.groebner_basis()
    print(B)
    for f in B:
        print('\n')
        print(f)
        d = deg_fall(f, f_list, ring)
        if d > f.degree():
            fall_degrees.append(d)
    if fall_degrees == []:
        return 0
    else:
        return max(fall_degrees)
    


In [142]:
# This is Example 5 in Caminata and Gorla

S.<x1,x2,x3> = PolynomialRing(GF(5), order = "lex")
f1 = x3^2 - x2
f2 = x2^3 - x1
I = ideal(f1,f2)
I.groebner_basis()

[x1 - x3^6, x2 - x3^2]

In [143]:
mac_grob([f1,f2],S)

0 [0] True False
1 [0] True False
2 [x2 - x3^2] True False
3 [x1*x2 - x1*x3^2, x1 - x2^3, x2^2 - x2*x3^2, x2*x3 - x3^3, x2 - x3^2] True True


(3, [x1*x2 - x1*x3^2, x1 - x2^3, x2^2 - x2*x3^2, x2*x3 - x3^3, x2 - x3^2])

In [144]:
sdeg([f1,f2],S)

2 [x2 - x3^2] True False
3 [x1*x2 - x1*x3^2, x1 - x2^3, x2^2 - x2*x3^2, x2*x3 - x3^3, x2 - x3^2] True True


(3, [x1*x2 - x1*x3^2, x1 - x2^3, x2^2 - x2*x3^2, x2*x3 - x3^3, x2 - x3^2])

In [123]:
mac_matrix([f1,f2],3,S).rref().row_space()

[x1^3, x1^2*x2, x1^2*x3, x1^2, x1*x2^2, x1*x2*x3, x1*x2, x1*x3^2, x1*x3, x1, x2^3, x2^2*x3, x2^2, x2*x3^2, x2*x3, x2, x3^3, x3^2, x3, 1]


Vector space of degree 20 and dimension 5 over Finite Field of size 5
Basis matrix:
[0 0 0 0 0 0 1 4 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 4 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 4 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 4 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 4 0 0]

In [124]:
d = 3
mons0 = monomials(list(S.gens()), [d+1 for i in range(len(S.gens()))])
mons = []
for mon in mons0:
    if mon.degree() <= d:
        mons.append(mon)
mons.sort()
mons.reverse()
pol = x2^2 - x2*x3^2
[pol.monomial_coefficient(mon) for mon in mons]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 0, 0, 0, 0, 0, 0]

In [125]:
vector([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 0, 0, 0, 0, 0, 0]) in mac_matrix([f1,f2],3,S).rref().row_space()

[x1^3, x1^2*x2, x1^2*x3, x1^2, x1*x2^2, x1*x2*x3, x1*x2, x1*x3^2, x1*x3, x1, x2^3, x2^2*x3, x2^2, x2*x3^2, x2*x3, x2, x3^3, x3^2, x3, 1]


True

In [127]:
smdeg([f1,f2],S)

2 [-x2 + x3^2] True False


KeyboardInterrupt: 

In [13]:
# This is Example 5 in Caminata and Gorla, with the field equations

S.<x1,x2,x3> = PolynomialRing(GF(5).algebraic_closure(), order = "degrevlex")
f1 = x3^2 - x2
f2 = x2^3 - x1
f3 = x1^5 - x1
f4 = x2^5 - x2
f5 = x3^5 - x3
I = ideal(f1,f2,f3,f4,f5)
I.groebner_basis()

verbose 0 (3848: multi_polynomial_ideal.py, groebner_basis) Warning: falling back to very slow toy implementation.


[x2^3 - x2, x2^2*x3 - x3, x3^2 - x2, x1 - x2]

In [12]:
sdeg([f1,f2,f3,f4,f5], S)

[1]
0 [0] True False
[x1, x2, x3, 1]
1 [0] True False
[x1^2, x1*x2, x2^2, x1*x3, x2*x3, x3^2, x1, x2, x3, 1]
2 [x3^2 - x2] True False
[x1^3, x1^2*x2, x1*x2^2, x2^3, x1^2*x3, x1*x2*x3, x2^2*x3, x1*x3^2, x2*x3^2, x3^3, x1^2, x1*x2, x2^2, x1*x3, x2*x3, x3^2, x1, x2, x3, 1]
3 [x2^3 - x1, x1*x3^2 - x1*x2, x2*x3^2 - x2^2, x3^3 - x2*x3, x3^2 - x2] True False
[x1^4, x1^3*x2, x1^2*x2^2, x1*x2^3, x2^4, x1^3*x3, x1^2*x2*x3, x1*x2^2*x3, x2^3*x3, x1^2*x3^2, x1*x2*x3^2, x2^2*x3^2, x1*x3^3, x2*x3^3, x3^4, x1^3, x1^2*x2, x1*x2^2, x2^3, x1^2*x3, x1*x2*x3, x2^2*x3, x1*x3^2, x2*x3^2, x3^3, x1^2, x1*x2, x2^2, x1*x3, x2*x3, x3^2, x1, x2, x3, 1]
4 [x1*x2^3 - x1^2, x2^4 - x1*x2, x2^3*x3 - x1*x3, x1^2*x3^2 - x1^2*x2, x1*x2*x3^2 - x1*x2^2, x2^2*x3^2 - x1, x1*x3^3 - x1*x2*x3, x2*x3^3 - x2^2*x3, x3^4 - x2^2, x2^3 - x1, x1*x3^2 - x1*x2, x2*x3^2 - x2^2, x3^3 - x2*x3, x3^2 - x2] True False
[x1^5, x1^4*x2, x1^3*x2^2, x1^2*x2^3, x1*x2^4, x2^5, x1^4*x3, x1^3*x2*x3, x1^2*x2^2*x3, x1*x2^3*x3, x2^4*x3, x1^3*x3^2, x1^2*x2

(5, Polynomial Sequence with 51 Polynomials in 3 Variables)

In [116]:
S.<x,y,z,u,v,w> = PolynomialRing(QQ, order = 'degrevlex')
I = Ideal(u^2*v*w + u^2, u*v^2*w + v^2 + 1, u*v*w^2 + w^2, x^2*y*z + x^2, x*y^2*z + y^2 + 1, x*y*z^2 + z^2)
mac_grob([u^2*v*w + u^2, u*v^2*w + v^2 + 1, u*v*w^2 + w^2, x^2*y*z + x^2, x*y^2*z + y^2 + 1, x*y*z^2 + z^2],S)

0 [0] True False
1 [0] True False
2 [0] True False
3 [0] True False
4 [x^2*y*z + x^2, x*y^2*z + y^2 + 1, x*y*z^2 + z^2, u^2*v*w + u^2, u*v^2*w + v^2 + 1, u*v*w^2 + w^2] False True
5 Polynomial Sequence with 42 Polynomials in 6 Variables False True
6 Polynomial Sequence with 168 Polynomials in 6 Variables False True
7 Polynomial Sequence with 502 Polynomials in 6 Variables False True
8 Polynomial Sequence with 1218 Polynomials in 6 Variables True True


(8, Polynomial Sequence with 1218 Polynomials in 6 Variables)

In [117]:
S.<x,y,z,u,v,w> = PolynomialRing(QQ, order = 'degrevlex')
sdeg([u^2*v*w + u^2, u*v^2*w + v^2 + 1, u*v*w^2 + w^2, x^2*y*z + x^2, x*y^2*z + y^2 + 1, x*y*z^2 + z^2],S)

4 [u^2*v*w + u^2, u*v^2*w + v^2 + 1, u*v*w^2 + w^2, x^2*y*z + x^2, x*y^2*z + y^2 + 1, x*y*z^2 + z^2] False True
5 Polynomial Sequence with 42 Polynomials in 6 Variables False True
6 Polynomial Sequence with 168 Polynomials in 6 Variables False True
7 Polynomial Sequence with 504 Polynomials in 6 Variables False True


KeyboardInterrupt: 

In [9]:
# Example from Minko's Thesis
R.<x,y,z> = PolynomialRing(GF(7), order = 'degrevlex')
f1 = x^5 + y^5 + z^5 - 1
f2 = x^3 + y^3 + z^2 - 1
f3 = y^6 - 1
f4 = z^6 - 1
f_list = [f1*f1*f1, f1*f1*f2, f1*f1*f3, f1*f1*f4, f1*f2*f2, f1*f2*f3, f1*f2*f4, f1*f3*f3, f1*f3*f4, f1*f4*f4, f2*f2*f2, f2*f2*f3, f2*f2*f4, f2*f3*f3, f2*f3*f4, f2*f4*f4, f3*f3*f3, f3*f3*f4, f3*f4*f4, f4*f4*f4, x^7 - x, y^7 - y, z^7 - z]
I = Ideal(f_list)
I.groebner_basis()

[y^6 - 1, y^2*z - y^2 - x*z + y*z - 3*x + 2*y + z - 1, x^2 - y^2 + 2*y*z - 2*x + 3*y - 3*z + 3, x*y + y^2 - x*z - 2*y*z + 3*x - 3*y + 3*z - 3, z^2 + 2*x + 2*y - 2*z + 1]

In [146]:
sdeg(f_list, R)

7 [x^7 - x, y^7 - y, z^7 - z] True False
8 [x^8 - x^2, x^7*y - x*y, x*y^7 - x*y, y^8 - y^2, x^7*z - x*z, y^7*z - y*z, x*z^7 - x*z, y*z^7 - y*z, z^8 - z^2, x^7 - x, y^7 - y, z^7 - z] True False
9 Polynomial Sequence with 31 Polynomials in 3 Variables False False
10 Polynomial Sequence with 64 Polynomials in 3 Variables False False
11 Polynomial Sequence with 116 Polynomials in 3 Variables False False
12 Polynomial Sequence with 194 Polynomials in 3 Variables False False
13 Polynomial Sequence with 304 Polynomials in 3 Variables False False
14 Polynomial Sequence with 450 Polynomials in 3 Variables False False
15 Polynomial Sequence with 636 Polynomials in 3 Variables False False
16 Polynomial Sequence with 862 Polynomials in 3 Variables False False
17 Polynomial Sequence with 1102 Polynomials in 3 Variables False False
18 Polynomial Sequence with 1304 Polynomials in 3 Variables False True
19 Polynomial Sequence with 1521 Polynomials in 3 Variables False True
20 Polynomial Sequence with 

(22, Polynomial Sequence with 2290 Polynomials in 3 Variables)

In [147]:
# Example from Minko's Thesis -- unclear what is going on here
R.<x,y,z> = PolynomialRing(GF(7), order = 'degrevlex')
f1 = x^5 + y^5 + z^5 - 1
f2 = x^3 + y^3 + z^2 - 1
f3 = x^7 - x
f4 = y^7 - y
f5 = z^7 - z
f_list = [f1*f1, f1*f2, f1*f3, f1*f4, f1*f5, f2*f2, f2*f3, f2*f4, f2*f5, f3*f3, f3*f4, f3*f5, f4*f4, f4*f5, f5*f5, x^7 - x, y^7 - y, z^7 - z]
I = Ideal(f_list)
I.groebner_basis()

[y^7 - y, y^3*z - y^3 + y^2*z + 3*x*y + 2*y^2 + 2*x*z + 3*y*z + 2*z^2 - x - 2*y - 3*z + 1, y^2*z^2 + 2*y^2*z - 2*y*z^2 - x*y + 3*y^2 + 3*x*z + y*z + 3*z^2 + 2*x - y - z - 2, x*y^2 + y^3 - 2*y^2*z - 3*y*z^2 + 3*x*y - 2*y^2 + 2*x*z - 3*y*z + 2*z^2 - x - 3*z + 1, x*y*z - x*y + 2*x*z + 2*y*z + 2*z^2 - x - y - 3*z + 1, x*z^2 + y*z^2 + 2*x*z + 2*y*z - 2*z^2 + 3*x + 3*y - 2*z - 3, z^3 + 2*x*z + 2*y*z - 2*z^2 + z, x^2 + 2*x*y + y^2 - 2*x*z - 2*y*z - 3*z^2 - 2*x - 2*y + 2*z + 1]

In [2]:
sdeg(f_list, R)

NameError: name 'f_list' is not defined

In [5]:
R.<x,y> = PolynomialRing(GF(5), order = 'degrevlex')
f1 = x*y + y
f2 = y^2 - 1
f3 = x^4 - 1
I = Ideal(f1,f2,f3)
I.groebner_basis()
sdeg([f1,f2,f3],R)

nothing
2 [x*y + y, y^2 - 1] False True
nothing
3 [x^2*y - y, x*y^2 + 1, y^3 - y, x*y + y, y^2 - 1, x + 1] True True


(3, [x^2*y - y, x*y^2 + 1, y^3 - y, x*y + y, y^2 - 1, x + 1])